<a href="https://colab.research.google.com/github/cengizmehmet/BenchmarkNets/blob/main/data_cleaner/SPEC2017_Cleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PREPROCESSING OF THE SPEC CPU2017 DATASET**

**Prepared by Mehmet CENGIZ**

ORCID: 0000-0003-4972-167X

As we use this format of the SPEC2017 dataset in our studies, we modify the original dataset based on our requirements. Those who will use this script is free to modify this adhering to their needs.



---



This script is prepared for making operable the SPEC2017 dataset to use in our further studies. Even though the dataset was prepared carefully by practitioners, there are many unnecessary and untidy data in it. We will explain every process applied on columns onwards.

## **NECESSARY DEPENDENCIES AND LIBRARIES**

In [ ]:
import pandas as pd
import re
from typing import Tuple
import enum

**Versions:** This information is the library versions in Google Colab when the models were first designed (around the end of 2022). Version differences may occur due to time and programming environment changes.

* Python: 3.8.16
* Pandas: 1.3.5
* Regular expression: 2.2.1



---



In [ ]:
path = 'https://raw.githubusercontent.com/cengizmehmet/BenchmarkNets/main/data/SPEC2017_Original.csv'

In [ ]:
dataset = pd.read_csv(path)

## **THE DATASET BEFORE PREPROCESSING**

After preprocessing, the dataset will be changed in many ways. In the section below, we present the original formats of some columns of the dataset.

**Column names:**

In [ ]:
print(len(dataset.columns))
print(dataset.columns)

**Data types:**

In [ ]:
dataset.dtypes

**Shape:**

In [ ]:
dataset.shape

**The dataset itself:**

Colab's dataset formatter is not working properly, because of so many columns and rows. The limit is 20 columns and 20000 rows. Those who want to see the dataset tidier and more interactive may click the baton symbol left down corner after running the following line.

In [ ]:
dataset

## The Common Functions

Some functions below require a column name as the parameter. In order to prevent errors from passing in invalid constants, we have defined an enumeration class.

In [ ]:
class Column_Names(enum.Enum):
  benchmark = "Benchmark"
  vendor = "Hardware_Vendor"
  system = "System"
  cores = "#_Cores"
  chips = "#_Chips"
  threads = "#_Enabled_Threads_Per_Core"
  processor = "Processor"
  mhz = "Processor_MHz"
  cpus = "CPU(s)_Orderable"
  parallel = "Parallel"
  base_pointer = "Base_Pointer_Size"
  peak_pointer = "Peak_Pointer_Size"
  first_cache = "1st_Level_Cache"
  second_cache = "2nd_Level_Cache"
  third_cache = "3rd_Level_Cache"
  other_cache = "Other_Cache"
  memory = "Memory"
  storage = "Storage"
  os = "Operating_System"
  file_system = "File_System"
  compiler = "Compiler"
  hw = "HW_Avail"
  sw = "SW_Avail"
  license = "License"
  tested_by = "Tested_By"
  sponsor = "Test_Sponsor"
  date = "Test_Date"
  published = "Published"
  updated = "Updated"
  disclosure = "Disclosures"
  result = "Base_Result"
  energy = "Energy_Base_Result"

**Method Name:** drop_empties

**Parameters:** pd.DataFrame, Column_Names, str

**Return:** pd.DataFrame

This function deletes the rows containing the character entered according to the specified column. e.g. delete rows with value of 0 in the Result column.

In [ ]:
def drop_empties(dataset: pd.DataFrame, column_name: Column_Names, empty_character: str) -> pd.DataFrame:
  dataset.drop(dataset.index[dataset[column_name.value] == empty_character], inplace = True)
  return dataset

**Method Name:** remove_between_with_delimiter

**Parameters:** pd.DataFrame, str, str

**Return:** pd.DataFrame

This function removes all characters between two delimiters including delimiters from a string. e. g.
> input: Test system (test info)

> delimiters: "(" and ")"

> output: Test system

In [ ]:
def remove_between_with_delimiter(column: pd.DataFrame.columns, delimiter1: str, delimiter2: str) -> pd.DataFrame.columns:
  column = list(column)
  delimiter1 = "\\" + delimiter1
  delimiter2 = "\\" + delimiter2
  pattern = pattern = delimiter1 + ".*?" + delimiter2
  count = 0
  while count < len(column):
    column[count] = re.sub(pattern, "", column[count])
    count += 1
  column = [s.strip() for s in column]
  return column

**Method Name:** remove_after_with_delimiter

**Parameters:** pd.DataFrame, str, str

**Return:** pd.DataFrame

This function removes all characters after a delimiter including the delimiter from a string. e. g.
> input: "Test system - test info"

> delimiter: "-"

> output: "Test system"

In [ ]:
def remove_after_with_delimiter(column: pd.DataFrame.columns, delimiter: str) -> pd.DataFrame.columns:
  column = list(column)
  count = 0
  while count < len(column):
    column[count] = column[count].split(delimiter, 1)[0]
    count += 1
  column = [s.strip() for s in column]
  return column

**Method Name:** eliminate_non_digits

**Parameters:** str

**Return:** int

This function removes all non-digit characters from a string and returns an integer. e. g.
> input: 45 kb info

> output: 45

In [ ]:
def eliminate_non_digits(text: str) -> int:
  numeric_filter = filter(str.isdigit, text)
  digits = "".join(numeric_filter)
  if digits == "":
    return 0
  else:
    return int(digits)

**Method Name:** find_anagrams

**Parameters:** pd.DataFrame.columns

**Return:** Tuple[list, list]

This function was designed to find the whole combinations of a string. The main goal is to find duplicates in columns such as "Pentium dual core e2140" and "Pentium dualcore e2140". The main approach is to find anagrams.

In [ ]:
def find_anagrams(column: pd.DataFrame.columns) -> Tuple[list, list]:
  values = list(set(column))
  checked = [False for i in range(len(values))]
  originals = []
  anagrams = []
  i = 0
  while i < len(values):
    if checked[i] == False:
      checked[i] = True
      str1 = sorted((values[i].replace("-", "")).replace(" ",""))
      j = i + 1
      while j < len(values):
        if checked[j] == False:
          str2 = sorted((values[j].replace("-", "")).replace(" ",""))
          if str1 == str2:
            checked[j] = True
            originals.append(values[i])
            anagrams.append(values[j])
            values[j] == values[i]          
        j += 1
    i += 1
  return originals, anagrams

## **PRE CLEARING**

**Clearing column names**

Column names have white spaces between words. In order to reach them easily in the script, we add "_" between words and trim each other. By doing this, a naming convention has been created.

In [ ]:
def clean_column_names(dataset):
  column_names = dataset.columns
  column_names = [s.strip() for s in column_names]
  column_names = [s.replace(' ', '_') for s in column_names]
  dataset.columns = column_names
  return dataset

In [ ]:
dataset = clean_column_names(dataset)

In [ ]:
dataset.columns



---



**Clearing rows**

The dataset contains some inconclusive tests. Result values were entered as 0 in these tests by the practitioners. First, we started by eliminating these lines.

In [ ]:
empty_char = 0
dataset = drop_empties(dataset, Column_Names.result, empty_char)



---



## **COLUMNS THAT DO NOT NEED TO BE CHANGE**

The columns below can be used as they are because their contents are clear. To provide a standard, we turned the string ones into lower cases.

*   Benchmark
*   #_Cores
*   #_Chips
*   #_Enabled_Threads_Per_Core
*   Processor
*   Processor MHz
*   Parallel
*   Base_Pointer_Size
*   Peak_Pointer_Size
*   1st Level Cache
*   Other Cache
*   Operating System
*   File System
*   Compiler
*   HW_Avail
*   SW_Avail
*   License
*   Test Date
*   Published
*   Updated
*   Disclosures

In [ ]:
dataset[Column_Names.benchmark.value] = dataset[Column_Names.benchmark.value].str.lower()
dataset[Column_Names.benchmark.value] = dataset[Column_Names.benchmark.value].str.strip()

dataset[Column_Names.processor.value] = dataset[Column_Names.processor.value].str.lower()
dataset[Column_Names.processor.value] = dataset[Column_Names.processor.value].str.strip()

dataset[Column_Names.parallel.value] = dataset[Column_Names.parallel.value].str.lower()
dataset[Column_Names.parallel.value] = dataset[Column_Names.parallel.value].str.strip()

dataset[Column_Names.base_pointer.value] = dataset[Column_Names.base_pointer.value].str.lower()
dataset[Column_Names.base_pointer.value] = dataset[Column_Names.base_pointer.value].str.strip()

dataset[Column_Names.peak_pointer.value] = dataset[Column_Names.peak_pointer.value].str.lower()
dataset[Column_Names.peak_pointer.value] = dataset[Column_Names.peak_pointer.value].str.strip()

dataset[Column_Names.first_cache.value] = dataset[Column_Names.first_cache.value].str.lower()
dataset[Column_Names.first_cache.value] = dataset[Column_Names.first_cache.value].str.strip()

dataset[Column_Names.other_cache.value] = dataset[Column_Names.other_cache.value].str.lower()
dataset[Column_Names.other_cache.value] = dataset[Column_Names.other_cache.value].str.strip()

dataset[Column_Names.os.value] = dataset[Column_Names.os.value].str.lower()
dataset[Column_Names.os.value] = dataset[Column_Names.os.value].str.strip()

dataset[Column_Names.file_system.value] = dataset[Column_Names.file_system.value].str.lower()
dataset[Column_Names.file_system.value] = dataset[Column_Names.file_system.value].str.strip()

dataset[Column_Names.compiler.value] = dataset[Column_Names.compiler.value].str.lower()
dataset[Column_Names.compiler.value] = dataset[Column_Names.compiler.value].str.strip()

dataset[Column_Names.hw.value] = dataset[Column_Names.hw.value].str.lower()
dataset[Column_Names.hw.value] = dataset[Column_Names.hw.value].str.strip()

dataset[Column_Names.sw.value] = dataset[Column_Names.sw.value].str.lower()
dataset[Column_Names.sw.value] = dataset[Column_Names.sw.value].str.strip()

dataset[Column_Names.date.value] = dataset[Column_Names.date.value].str.lower()
dataset[Column_Names.date.value] = dataset[Column_Names.date.value].str.strip()

dataset[Column_Names.published.value] = dataset[Column_Names.published.value].str.lower()
dataset[Column_Names.published.value] = dataset[Column_Names.published.value].str.strip()

dataset[Column_Names.updated.value] = dataset[Column_Names.updated.value].str.lower()
dataset[Column_Names.updated.value] = dataset[Column_Names.updated.value].str.strip()

dataset[Column_Names.disclosure.value] = dataset[Column_Names.disclosure.value].str.lower()
dataset[Column_Names.disclosure.value] = dataset[Column_Names.disclosure.value].str.strip()

The columns "Result" and "Baseline" are the possible target attributes. Therefore we do not change them.

## **COLUMNS TO BE CORRECTED**

### **HARDWARE VENDOR**

Originally, there are 59 different hardware vendors. However, after we cleaned the rows containing 0 values, the number decreased to 53.

In [ ]:
dataset[Column_Names.vendor.value] = dataset[Column_Names.vendor.value].str.lower()
dataset[Column_Names.vendor.value] = dataset[Column_Names.vendor.value].str.strip()
vendors = set(list(dataset[Column_Names.vendor.value]))
print(len(vendors))
print(sorted(vendors))

In the column, there are duplicated vendors such as "giga-byte technology co., ltd" and "giga-byte technology co., ltd." -the diffrence is the punctuation at the end. We need to make them one. It is a quite straightforward process, there is no smart approach. However, in order to not lose any vendor the coding order is important.

In [ ]:
dataset[Column_Names.vendor.value] = remove_after_with_delimiter(dataset[Column_Names.vendor.value], "incorporated")
dataset[Column_Names.vendor.value] = remove_after_with_delimiter(dataset[Column_Names.vendor.value], "computing")
dataset[Column_Names.vendor.value] = remove_after_with_delimiter(dataset[Column_Names.vendor.value], "computer")
dataset[Column_Names.vendor.value] = remove_after_with_delimiter(dataset[Column_Names.vendor.value], "systems")
dataset[Column_Names.vendor.value] = remove_after_with_delimiter(dataset[Column_Names.vendor.value], "inc.")
dataset[Column_Names.vendor.value] = remove_after_with_delimiter(dataset[Column_Names.vendor.value], "sp.")
dataset[Column_Names.vendor.value] = remove_after_with_delimiter(dataset[Column_Names.vendor.value], "technology")
dataset[Column_Names.vendor.value] = remove_after_with_delimiter(dataset[Column_Names.vendor.value], "corporation")
dataset[Column_Names.vendor.value] = remove_after_with_delimiter(dataset[Column_Names.vendor.value], "enterprise")
dataset[Column_Names.vendor.value] = remove_after_with_delimiter(dataset[Column_Names.vendor.value], "global")
dataset[Column_Names.vendor.value] = remove_after_with_delimiter(dataset[Column_Names.vendor.value], "technologies")
dataset[Column_Names.vendor.value] = dataset[Column_Names.vendor.value].replace("new h3c", "h3c")
dataset[Column_Names.vendor.value] = remove_after_with_delimiter(dataset[Column_Names.vendor.value], "cloud")

Three exceptions:


1.   M Computers' name must be still as it is.
2.   Sun Microsystems is also known as Sun. For further processes, we decided to use it as Sun.
3.   AsusTek is also known as Asus.

In [ ]:
i = 0
hw_vendors = list(dataset[Column_Names.vendor.value])
while i < len(dataset[Column_Names.vendor.value]):
  if hw_vendors[i] == "m":
    hw_vendors[i] = "m computers"
  elif hw_vendors[i] == "sun micro":
    hw_vendors[i] = "sun"
  elif hw_vendors[i] == "asustek":
    hw_vendors[i] = "asus"
  i += 1
dataset[Column_Names.vendor.value] = hw_vendors

As a result, the number of vendors downs to 34. Duplicates and unnecessary parts were eliminated.

In [ ]:
vendors = set(list(dataset['Hardware_Vendor']))
print(len(vendors))
print(sorted(vendors))

### **ORDERABLE CPU(s)**

The `Orderable_CPU(s)` column has duplicated values such as "1,2 chips" and "1,2 chip(s)". Also, it contains blank values.

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].str.lower()
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].str.strip()
cpus = set(list(dataset[Column_Names.cpus.value].astype(str)))
print(len(cpus))
print(sorted(cpus))

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].str.replace(" ", "")

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].str.replace(",", "-")

`1 chip`:

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1chip", "1 chip")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1chip(s)", "1 chip")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1chips", "1 chip")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1cpuchip;2-3-4-..6cores", "1 chip")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1xhip", "1 chip")

`1-2` chips:

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-2chips", "1-2 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-2", "1-2 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-2(chip)s", "1-2 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-2(chips)s", "1-2 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-2chip", "1-2 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-2chip(s)", "1-2 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1.2chips", "1-2 chips")

`1-4 chips`:

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-2-3-4chip", "1-4 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-2-3-4chip(s)", "1-4 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-2-3-4chips", "1-4 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-4chips", "1-4 chips")

`1-8 chips`:

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-2-3-4-5-6-7-8chips", "1-8 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-8chips", "1-8 chips")

`1-16 chips`, `1-16 cmious`, `1-24 chips`, and `1-32 chips`:

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-16chips", "1-16 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-16cmiou(onhost)", "1-16 cmiou")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-24chips", "1-24 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-32chips", "1-32 chips")

`1,2,4 chips`:

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-2-4(chip)s", "1,2,4 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-2-4chip(s)", "1,2,4 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("1-2-4chips", "1,2,4 chips")

`2 chips`:

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2chip", "2 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2chips", "2 chips")

`2-4 chips`:

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2-3-4chips", "2-4 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2-4", "2-4 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2-4chips", "2-4 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2-4chip(s)", "2-4 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2-4chip", "2-4 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2or4chips", "2-4 chips")

`2-4-8 chips`, `2-4-6-8 chips`, `2-3-4-6-8 chips`, and `2-4-8-16 chips`:

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2-4-8chip(s)", "2-4-8 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2-4-8chips", "2-4-8 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2-4-6-8chips", "2-4-6-8 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2-3-4-6-8chips", "2-3-4-6-8 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2-4-8-16chip", "2-4-8-16 chips")

`2-16 chips`:

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("2to16chips", "2-16 chips")

`3-4 chips`:

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("3-4chips", "3-4 chips")

`4 chips`:

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("4chips", "4 chips")

`4-8 chips`, `4-8-16 chips`, and `4-32 chips`:

In [ ]:
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("4-8chips", "4-8 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("4-8-16chips", "4-8-16 chips")
dataset[Column_Names.cpus.value] = dataset[Column_Names.cpus.value].replace("4to32chips", "4-32 chips")

### **CACHES**

In the dataset, there are 4 types of caches: 1st Level, 2nd Level, 3rd Level, and Other. Except for the first level and other caches, they need to be corrected in different ways.

#### **2nd Level Cache**

There are 7 different types of 2nd level caches. Only the "2 mb i on chip per chip (256 kb / 4 cores); 4 mb d on chip per chip (256 kb / 2 cores)" must be corrected.

In [ ]:
dataset[Column_Names.second_cache.value] = dataset[Column_Names.second_cache.value].str.lower()
dataset[Column_Names.second_cache.value] = dataset[Column_Names.second_cache.value].str.strip()

In [ ]:
second_level_caches = set(list(dataset[Column_Names.second_cache.value]))
print(len(second_level_caches))
print(second_level_caches)

In [ ]:
dataset[Column_Names.second_cache.value] = dataset[Column_Names.second_cache.value].replace(
    "2 mb i on chip per chip (256 kb / 4 cores); 4 mb d on chip per chip (256 kb / 2 cores)",
    "2 mb i on chip per chip + 4 mb d on chip per chip")

#### **3rd Level Cache**

There are 74 different types of 3rd level caches. We cleaned up some information that seems unnecessary. Fortunately, information such as that was mostly stored after the comma.

In [ ]:
dataset[Column_Names.third_cache.value] = dataset[Column_Names.third_cache.value].str.lower()
dataset[Column_Names.third_cache.value] = dataset[Column_Names.third_cache.value].str.strip()

In [ ]:
third_level_caches = set(list(dataset[Column_Names.third_cache.value]))
print(len(third_level_caches))
print(third_level_caches)

In [ ]:
dataset[Column_Names.third_cache.value] = remove_after_with_delimiter(dataset[Column_Names.third_cache.value], ",")
dataset[Column_Names.third_cache.value] = remove_after_with_delimiter(dataset[Column_Names.third_cache.value], "shared")
dataset[Column_Names.third_cache.value] = remove_after_with_delimiter(dataset[Column_Names.third_cache.value], "(")

### **SYSTEM**

In many cases, system names involve both the vendor's name and the processor specs. We have to get rid of them because that information has its own columns.

**Method Name:** remove_vendor_names

**Parameters:** pd.DataFrame.columns, pd.DataFrame.columns

**Return:** pd.DataFrame.columns

This function deletes vendor names in system names.

In [ ]:
def remove_vendor_names(systems: pd.DataFrame.columns, vendors: pd.DataFrame.columns) -> pd.DataFrame.columns:
  vendors = list(set(list(vendors)))
  systems = list(systems)
  i = 0
  while i < len(systems):
    j = 0
    temp = systems[i].strip()
    while j < len(vendors):
      if vendors[j] in temp:
        start_index = systems[i].find(vendors[j])
        end_index = start_index + len(vendors[j])
        temp = temp[0:start_index:] + temp[end_index::]
        systems[i] = temp.strip()
        break
      j +=1
    i += 1
  return systems

In [ ]:
dataset[Column_Names.system.value] = dataset[Column_Names.system.value].str.lower()
dataset[Column_Names.system.value] = dataset[Column_Names.system.value].str.strip()

In [ ]:
systems = set(list(dataset[Column_Names.system.value]))

In [ ]:
dataset[Column_Names.system.value] = remove_after_with_delimiter(dataset[Column_Names.system.value], "(")
dataset[Column_Names.system.value] = remove_after_with_delimiter(dataset[Column_Names.system.value], ",")
dataset['System'] = remove_vendor_names(dataset[Column_Names.system.value], dataset[Column_Names.vendor.value])
dataset[Column_Names.system.value] = remove_after_with_delimiter(dataset[Column_Names.system.value], "2.")
dataset[Column_Names.system.value] = remove_after_with_delimiter(dataset[Column_Names.system.value], "3.")
dataset[Column_Names.system.value] = remove_after_with_delimiter(dataset[Column_Names.system.value], "intel")
dataset[Column_Names.system.value] = remove_after_with_delimiter(dataset[Column_Names.system.value], "amd")

In [ ]:
dataset['System'] = dataset[Column_Names.system.value].str.strip()
dataset = drop_empties(dataset, Column_Names.system, "")

After eliminating redundancy from the dataset, we have to manage typos. In some cases, some system names were entered with different ways such as "yr190b8228" and "yr190 b8228".

To solve this problem, we designed a method that finds anagrams as mentioned earlier. However, this cannot help us totally. Because there are real anagrams in the dataset such as "proliant ml350 gen 10" and "proliant ml350 gen10". 

On the other hand, with the help of anagrams, we narrowed the search space down. The anagram finder finds 47 anagrams.

In [ ]:
original_systems, system_anagrams = find_anagrams(dataset['System'])

And we eliminate duplicated and typoed entries manually selecting from the list below.

In [ ]:
i = 0
while i < len(system_anagrams):
  print(original_systems[i] + " - " + system_anagrams[i])
  i += 1

As a result, we corrected 15 different system entries manually as below.

In [ ]:
dataset[Column_Names.system.value] = dataset[Column_Names.system.value].replace("proliant ml350 gen 10", "proliant ml350 gen10")
dataset[Column_Names.system.value] = dataset[Column_Names.system.value].replace("8100v5", "8100 v5")
dataset[Column_Names.system.value] = dataset[Column_Names.system.value].replace("poweredge r750 xa", "poweredge r750xa")
dataset[Column_Names.system.value] = dataset[Column_Names.system.value].replace("prolaint dl385 gen10", "proliant dl385 gen10")

### **MEMORY**

In the dataset, the Memory column contains not only the size of the system memory but also the combination of memory components such as "32 GB (4 x 8 GB 2Rx8 PC3-12800E-11, ECC)". Also, because of TB to GB conversions, some of them indicate the same size. So we cleaned them up.

In [ ]:
dataset[Column_Names.memory.value] = dataset[Column_Names.memory.value].str.lower()
dataset[Column_Names.memory.value] = dataset[Column_Names.memory.value].str.strip()

In [ ]:
dataset[Column_Names.memory.value] = remove_after_with_delimiter(dataset[Column_Names.memory.value], "(")
dataset[Column_Names.memory.value] = dataset[Column_Names.memory.value].str.strip()

In [ ]:
memories = set(list(dataset[Column_Names.memory.value]))

In [ ]:
memories

After eliminating non-essential parts, from TB and GB to MB conversions is applied. As cache sizes, we use the thousand systems: 1 TB = 1000 GB and 1 GB = 1000 MB.

In [ ]:
column = list(dataset[Column_Names.memory.value])
i = 0
while i < len(column):
  if "gb" in column[i]:
    x = eliminate_non_digits(column[i])
    column[i] = 1000 * eliminate_non_digits(column[i])
  else:
    x = eliminate_non_digits(column[i])
    column[i] = 1000000 * eliminate_non_digits(column[i])
  i += 1
dataset[Column_Names.memory.value] = column

### **STORAGE**

There 426 different types of storages in the dataset. As a result of the following actions, we left behind the dimensions of the storage areas rather than their specifications.

In [ ]:
dataset[Column_Names.storage.value] = dataset[Column_Names.storage.value].str.lower()
dataset[Column_Names.storage.value] = dataset[Column_Names.storage.value].str.strip()

In [ ]:
storage = set(list(dataset[Column_Names.storage.value]))

In [ ]:
storage

In [ ]:
dataset[Column_Names.storage.value] = dataset[Column_Names.storage.value].str.replace(" ", "")

In [ ]:
column = list(dataset[Column_Names.storage.value])
i = 0
plain_pattern = r'[0-9]*\.?[0-9]+[gt]b'
pattern_with_x = r'^[0-9]+x[0-9]+\.?[0-9]*[gt]b'
typo_pattern = r'[0-9]*\.?[0-9]+[gt]'
while i < len(column):
  temp = re.findall(pattern_with_x, column[i])
  if temp:
    mult, data = temp[0].split("x")
    mult = float(mult)
    modifier = data[len(data) - 2 : len(data)]
    size = float(data[ : len(data) - 2])
    column[i] = mult * size
    if modifier == "tb":
        column[i] *= 1000
  else:
    temp = re.findall(plain_pattern, column[i])
    if temp:
      temp = temp[0]
      modifier = temp[len(temp) - 2 : len(temp)]
      size = float(temp[ : len(temp) - 2])
      column[i] = size
      if modifier == "tb":
        column[i] *= 1000
    else:
      temp = re.findall(typo_pattern, column[i])
      if temp:
        temp = temp[0]
        modifier = temp[len(temp) - 1 : len(temp)]
        size = float(temp[ : len(temp) - 1])
        column[i] = size
        if modifier == "tb":
          column[i] *= 1000
  i += 1
dataset[Column_Names.storage.value] = column

### **TESTED BY**

The tests in the dataset were applied by 36 different practitioners. However, like the `Hardware_Vendor` column, there are duplicated values.

In [ ]:
dataset[Column_Names.tested_by.value] = dataset[Column_Names.tested_by.value].str.lower()
dataset[Column_Names.tested_by.value] = dataset[Column_Names.tested_by.value].str.strip()

In [ ]:
tested_by = set(list(dataset[Column_Names.tested_by.value]))

In [ ]:
tested_by

In [ ]:
dataset[Column_Names.tested_by.value] = remove_after_with_delimiter(dataset[Column_Names.tested_by.value], "incorporated")
dataset[Column_Names.tested_by.value] = remove_after_with_delimiter(dataset[Column_Names.tested_by.value], "computing")
dataset[Column_Names.tested_by.value] = remove_after_with_delimiter(dataset[Column_Names.tested_by.value], "computer")
dataset[Column_Names.tested_by.value] = remove_after_with_delimiter(dataset[Column_Names.tested_by.value], "systems")
dataset[Column_Names.tested_by.value] = remove_after_with_delimiter(dataset[Column_Names.tested_by.value], "inc.")
dataset[Column_Names.tested_by.value] = remove_after_with_delimiter(dataset[Column_Names.tested_by.value], "sp.")
dataset[Column_Names.tested_by.value] = remove_after_with_delimiter(dataset[Column_Names.tested_by.value], "corporation")
dataset[Column_Names.tested_by.value] = remove_after_with_delimiter(dataset[Column_Names.tested_by.value], "technologies")
dataset[Column_Names.tested_by.value] = dataset[Column_Names.tested_by.value].replace("new h3c", "h3c")
dataset[Column_Names.tested_by.value] = remove_after_with_delimiter(dataset[Column_Names.tested_by.value], "global")
dataset[Column_Names.tested_by.value] = remove_after_with_delimiter(dataset[Column_Names.tested_by.value], "co.")

### **TEST SPONSOR**

Each test's sponsor is a different company/corporation. In total, there 36 different sponsors. However, like the `Hardware_Vendor` column, there are duplicated values.

In [ ]:
dataset[Column_Names.sponsor.value] = dataset[Column_Names.sponsor.value].str.lower()
dataset[Column_Names.sponsor.value] = dataset[Column_Names.sponsor.value].str.strip()

In [ ]:
sponsor = set(list(dataset[Column_Names.sponsor.value]))

In [ ]:
sponsor

In [ ]:
dataset[Column_Names.sponsor.value] = remove_after_with_delimiter(dataset[Column_Names.sponsor.value], "incorporated")
dataset[Column_Names.sponsor.value] = remove_after_with_delimiter(dataset[Column_Names.sponsor.value], "computing")
dataset[Column_Names.sponsor.value] = remove_after_with_delimiter(dataset[Column_Names.sponsor.value], "computer")
dataset[Column_Names.sponsor.value] = remove_after_with_delimiter(dataset[Column_Names.sponsor.value], "systems")
dataset[Column_Names.sponsor.value] = remove_after_with_delimiter(dataset[Column_Names.sponsor.value], "inc.")
dataset[Column_Names.sponsor.value] = remove_after_with_delimiter(dataset[Column_Names.sponsor.value], "sp.")
dataset[Column_Names.sponsor.value] = remove_after_with_delimiter(dataset[Column_Names.sponsor.value], "corporation")
dataset[Column_Names.sponsor.value] = remove_after_with_delimiter(dataset[Column_Names.sponsor.value], "technologies")
dataset[Column_Names.sponsor.value] = dataset[Column_Names.sponsor.value].replace("new h3c", "h3c")
dataset[Column_Names.sponsor.value] = remove_after_with_delimiter(dataset[Column_Names.sponsor.value], "global")
dataset[Column_Names.sponsor.value] = remove_after_with_delimiter(dataset[Column_Names.sponsor.value], "co.")

## **EXPORTING THE DATASET**

In [ ]:
dataset.to_excel(r'SPEC2017_modified.xlsx', index = False, header = True)